In [1]:
!pip install spark-nlp==1.8.2
#once done , restart the kernel and refresh and then the next cell

    100% |████████████████████████████████| 101.6MB 280kB/s eta 0:00:01 1% |▋                               | 1.9MB 28.7MB/s eta 0:00:04    5% |█▊                              | 5.6MB 25.1MB/s eta 0:00:04    9% |███                             | 9.3MB 21.0MB/s eta 0:00:05    11% |███▊                            | 11.8MB 28.8MB/s eta 0:00:04    12% |████▏                           | 13.1MB 28.1MB/s eta 0:00:04    14% |████▌                           | 14.4MB 22.2MB/s eta 0:00:04    15% |█████                           | 15.5MB 24.8MB/s eta 0:00:04    20% |██████▌                         | 20.6MB 25.4MB/s eta 0:00:04    28% |█████████                       | 28.6MB 23.7MB/s eta 0:00:04    34% |███████████▏                    | 35.4MB 21.6MB/s eta 0:00:04    37% |████████████                    | 37.8MB 25.8MB/s eta 0:00:03    40% |█████████████                   | 41.4MB 23.6MB/s eta 0:00:03    52% |████████████████▉               | 53.5MB 21.0MB/s eta 0:00:03    55% |█████████████████▊ 

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 800)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .config("spark.jars.packages", 'com.johnsnowlabs.nlp:spark-nlp_2.11:1.8.2')\
.getOrCreate()
spark

In [3]:
dataPath = './data/reddit/*.json'
df = spark.read.json(dataPath)
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)
 |  

In [5]:
dfAuthorTitle = df.select('data.title' ,'data.author')
dfAuthorTitle.limit(3).toPandas()

,title,author
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.",jaykirsch
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,canuck_burger
2,"Iranian ""morality police"" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.",honolulu_oahu_mod


In [6]:
import pyspark.sql.functions as F

In [13]:
from pyspark.sql.functions import desc

In [17]:
dfWordCount = df.select(F.explode(
    F.split('data.title', ' ')).alias('word')).groupBy('word').count().sort(desc('count'))
dfWordCount.limit(10).toPandas()

,word,count
0,to,58
1,the,46
2,of,42
3,in,41
4,a,25
5,for,20
6,and,19
7,from,12
8,on,11
9,over,10


In [18]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp

In [19]:
dfAnnotated = bp.annotate(dfAuthorTitle, 'title')
dfAnnotated.printSchema()

root
 |-- text: string (nullable = true)
 |-- author: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- normal: array (nullable = true)
 |    |-- element: struct (contains

In [20]:
dfPos = dfAnnotated.select('text', 'pos.metadata', 'pos.result')
dfPos.limit(5).toPandas()

,text,metadata,result
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.","[{'word': 'Microsoft'}, {'word': 'Corp'}, {'word': 'said'}, {'word': 'it'}, {'word': 'has'}, {'word': 'discovered'}, {'word': 'hacking'}, {'word': 'targeting'}, {'word': 'democratic'}, {'word': 'institutions'}, {'word': 'think'}, {'word': 'tanks'}, {'word': 'and'}, {'word': 'nonprofit'}, {'word': 'organizations'}, {'word': 'in'}, {'word': 'Europe'}]","[NNP, NNP, VBD, PRP, VBZ, VBN, VBG, VBG, JJ, NNS, VBP, NNS, CC, NN, NNS, IN, NNP]"
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,"[{'word': 'Deutsche'}, {'word': 'Bank'}, {'word': 'reportedly'}, {'word': 'planned'}, {'word': 'to'}, {'word': 'extend'}, {'word': 'the'}, {'word': 'dates'}, {'word': 'of'}, {'word': 'million'}, {'word': 'in'}, {'word': 'loans'}, {'word': 'to'}, {'word': 'Trump'}, {'word': 'Organization'}, {'word': 'to'}, {'word': 'avoid'}, {'word': 'a'}, {'word': 'potential'}, {'word': 'nightmare'}, {'word': 'of'}, {'word': 'chasing'}, {'word': 'a'}, {'word': 'sitting'}, {'word': 'president'}, {'word': 'for'}, {'word': 'cash'}]","[NNP, NNP, RB, VBD, TO, VB, DT, NNS, IN, CD, IN, NNS, TO, NNP, NNP, TO, VB, DT, JJ, NN, IN, VBG, DT, VBG, NN, IN, NN]"
2,"Iranian ""morality police"" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.","[{'word': 'Iranian'}, {'word': 'morality'}, {'word': 'police'}, {'word': 'were'}, {'word': 'forced'}, {'word': 'to'}, {'word': 'fire'}, {'word': 'warning'}, {'word': 'shots'}, {'word': 'when'}, {'word': 'a'}, {'word': 'crowd'}, {'word': 'intervened'}, {'word': 'to'}, {'word': 'prevent'}, {'word': 'them'}, {'word': 'from'}, {'word': 'arresting'}, {'word': 'two'}, {'word': 'women'}, {'word': 'for'}, {'word': 'not'}, {'word': 'wearing'}, {'word': 'a'}, {'word': 'hijab'}, {'word': 'The'}, {'word': 'incident'}, {'word': 'occurred'}, {'word': 'in'}, {'word': 'Tehran'}, {'word': 's'}, {'word': 'northeastern'}, {'word': 'Narmak'}, {'word': 'neighbourhood'}, {'word': 'on'}, {'word': 'Friday'}, {'word': 'night'}, {'word': 'and'}, {'word': 'ended'}, {'word': 'with'}, {'word': 'a'}, {'word': 'mob'...","[JJ, NN, NN, VBD, VBN, TO, VB, NN, NNS, WRB, DT, NN, VBD, TO, VB, PRP, IN, VBG, CD, NNS, IN, RB, VBG, DT, NN, DT, NN, VBD, IN, NNP, VBZ, JJ, NNP, NN, IN, NNP, NN, CC, VBD, IN, DT, NN, VBG, DT, NN, RP, DT, NN, NN]"
3,"Trump administration 'pushing Saudi nuclear deal' which could benefit company linked to Jared Kushner - Senior Trump administration officials pushed a project to share nuclear power technology with Saudi Arabia over the objections of ethics officials, according to a congressional report","[{'word': 'Trump'}, {'word': 'administration'}, {'word': 'pushing'}, {'word': 'Saudi'}, {'word': 'nuclear'}, {'word': 'deal'}, {'word': 'which'}, {'word': 'could'}, {'word': 'benefit'}, {'word': 'company'}, {'word': 'linked'}, {'word': 'to'}, {'word': 'Jared'}, {'word': 'Kushner'}, {'word': 'Senior'}, {'word': 'Trump'}, {'word': 'administration'}, {'word': 'officials'}, {'word': 'pushed'}, {'word': 'a'}, {'word': 'project'}, {'word': 'to'}, {'word': 'share'}, {'word': 'nuclear'}, {'word': 'power'}, {'word': 'technology'}, {'word': 'with'}, {'word': 'Saudi'}, {'word': 'Arabia'}, {'word': 'over'}, {'word': 'the'}, {'word': 'objections'}, {'word': 'of'}, {'word': 'ethics'}, {'word': 'officials'}, {'word': 'according'}, {'word': 'to'}, {'word': 'a'}, {'word': 'congressional'}, {'word': 're...","[NNP, NN, VBG, NNP, NN, NN, WDT, MD, VB, NN, VBN, TO, NNP, NNP, NNP, NNP, NN, NNS, VBD, DT, NN, TO, VB, JJ, NN, NN, IN, NNP, NNP, IN, DT, NNS, IN, NNS, NNS, VBG, TO, DT, JJ, NN

In [22]:
dfPos = dfAnnotated.select(F.explode('pos').alias('pos'))
dfPos.printSchema()

root
 |-- pos: struct (nullable = true)
 |    |-- annotatorType: string (nullable = true)
 |    |-- begin: integer (nullable = false)
 |    |-- end: integer (nullable = false)
 |    |-- result: string (nullable = true)
 |    |-- metadata: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



In [24]:
df_NNP_NNPS = dfPos.where("pos.result == 'NNP' or pos.result == 'NNPS'")
df_NNP_NNPS.limit(10).toPandas()

,pos
0,"(pos, 0, 8, NNP, {'word': 'Microsoft'})"
1,"(pos, 10, 13, NNP, {'word': 'Corp'})"
2,"(pos, 126, 131, NNP, {'word': 'Europe'})"
3,"(pos, 0, 7, NNP, {'word': 'Deutsche'})"
4,"(pos, 9, 12, NNP, {'word': 'Bank'})"
5,"(pos, 81, 85, NNP, {'word': 'Trump'})"
6,"(pos, 87, 98, NNP, {'word': 'Organization'})"
7,"(pos, 175, 180, NNP, {'word': 'Tehran'})"
8,"(pos, 197, 202, NNP, {'word': 'Narmak'})"
9,"(pos, 221, 226, NNP, {'word': 'Friday'})"


In [28]:
dfWordTag = df_NNP_NNPS.selectExpr("pos.metadata['word'] as word", "pos.result as tag")
dfWordTag.limit(8).toPandas()

,word,tag
0,Microsoft,NNP
1,Corp,NNP
2,Europe,NNP
3,Deutsche,NNP
4,Bank,NNP
5,Trump,NNP
6,Organization,NNP
7,Tehran,NNP


In [29]:
dfWordTag.createOrReplaceTempView('wordtag')
spark.sql("""
select word, count(*)
from wordtag
group by word
order by count(*) desc
limit 10
""").toPandas()

,word,count(1)
0,US,14
1,Trump,9
2,Saudi,8
3,Putin,7
4,Russia,6
5,Arabia,5
6,Europe,5
7,Catholic,4
8,UK,4
9,Vladimir,4


In [30]:
dfWordTag.groupby('word').count().sort(desc('count')).limit(10).toPandas()

,word,count
0,US,14
1,Trump,9
2,Saudi,8
3,Putin,7
4,Russia,6
5,Arabia,5
6,Europe,5
7,Catholic,4
8,UK,4
9,Vladimir,4
